In [1]:
from hypersurface_tf import *
from generate_h import *
from biholoNN import *
import tensorflow as tf
import numpy as np

### Prepare the dataset:

In [2]:
z0, z1, z2, z3, z4 = sp.symbols('z0, z1, z2, z3, z4')
Z = [z0,z1,z2,z3,z4]
f = z0**5 + z1**5 + z2**5 + z3**5 + z4**5 + 0.5*z0*z1*z2*z3*z4
np.random.seed(123)
HS = Hypersurface(Z, f, 10000)
np.random.seed(124)
HS_test = Hypersurface(Z, f, 10000)

In [3]:
train_set = generate_dataset(HS)
test_set = generate_dataset(HS_test)

In [4]:
train_set = train_set.shuffle(500000).batch(1000)
test_set = test_set.shuffle(50000).batch(1000)

### Build the model:

In [5]:
class KahlerPotential(tf.keras.Model):

    def __init__(self):
        super(KahlerPotential, self).__init__()
        self.biholomorphic = Biholomorphic()
        #self.layer1 = WidthOneDense()
        #self.layer1 = Dense(25, 1, activation=None)
        self.layer1 = Dense(25,500, activation=tf.square)
        self.layer2 = Dense(500,500, activation=tf.square)
        #self.layer3 = tf.keras.layers.Dense(100, activation=tf.square, use_bias=False)
        #self.layer4 = tf.keras.layers.Dense(100, activation=tf.square, use_bias=False)
        #self.layer5 = tf.keras.layers.Dense(100, activation=tf.square, use_bias=False)
        #self.g = ComplexG(70)
        
    def call(self, inputs):
        x = self.biholomorphic(inputs)
        x = self.layer1(x)
        x = self.layer2(x)
        #x = self.layer3(x)
        #x = self.layer4(x)
        #x = self.layer5(x)
        #x = self.g(x)
        #x = tf.linalg.diag_part(tf.matmul(x, x, adjoint_b=True))
        #x = tf.math.log(x)
        x = tf.reduce_sum(x, 1)
        x = tf.math.log(x)
        return x

In [6]:
model = KahlerPotential()

In [7]:
@tf.function
def volume_form(x, Omega_Omegabar, mass, restriction):

    kahler_metric = complex_hessian(tf.math.real(model(x)), x)
    volume_form = tf.math.real(tf.linalg.det(tf.matmul(restriction, tf.matmul(kahler_metric, restriction, adjoint_b=True))))
    weights = mass / tf.reduce_sum(mass)
    factor = tf.reduce_sum(weights * volume_form / Omega_Omegabar)
    #factor = tf.constant(35.1774, dtype=tf.complex64)
    return volume_form / factor

In [ ]:
optimizer = tf.keras.optimizers.Adam()
epochs = 6000

for epoch in range(epochs):
    for step, (points, Omega_Omegabar, mass, restriction) in enumerate(train_set):
        with tf.GradientTape() as tape:
            
            omega = volume_form(points, Omega_Omegabar, mass, restriction)
            loss = weighted_MAPE(Omega_Omegabar, omega, mass)  
            grads = tape.gradient(loss, model.trainable_weights)
            #print(model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

        if step % 500 == 0:
            print("step %d: loss = %.4f" % (step, loss))
    
    test_loss = 0
    test_loss_old = 100
    total_mass = 0
    
    for step, (points, Omega_Omegabar, mass, restriction) in enumerate(test_set):
        omega = volume_form(points, Omega_Omegabar, mass, restriction)
        mass_sum = tf.reduce_sum(mass)
        test_loss += weighted_MAPE(Omega_Omegabar, omega, mass) * mass_sum
        total_mass += mass_sum
   
    test_loss = test_loss / total_mass
    print("test_loss:", test_loss.numpy())
    
    # This part doesn't work right now
    #if test_loss > test_loss_old:
    #    break
    #test_loss_old = test_loss

step 0: loss = 0.3250
test_loss: 0.050744288
step 0: loss = 0.0524
test_loss: 0.03756032
step 0: loss = 0.0365
test_loss: 0.031715397
step 0: loss = 0.0307
test_loss: 0.027932554
step 0: loss = 0.0262
test_loss: 0.026691275
step 0: loss = 0.0267
test_loss: 0.024220778
step 0: loss = 0.0236
test_loss: 0.023064254
step 0: loss = 0.0225
test_loss: 0.02187635
step 0: loss = 0.0223
test_loss: 0.02144316
step 0: loss = 0.0213
test_loss: 0.02050266
step 0: loss = 0.0202
test_loss: 0.019202689
step 0: loss = 0.0183
test_loss: 0.019057862
step 0: loss = 0.0178
test_loss: 0.018288774
step 0: loss = 0.0182
test_loss: 0.018188179
step 0: loss = 0.0184
test_loss: 0.018325953
step 0: loss = 0.0182
test_loss: 0.016554203
step 0: loss = 0.0156
test_loss: 0.016850324
step 0: loss = 0.0160
test_loss: 0.016012104
step 0: loss = 0.0152
test_loss: 0.015910357
step 0: loss = 0.0150
test_loss: 0.015737806
step 0: loss = 0.0140
test_loss: 0.015022068
step 0: loss = 0.0142
test_loss: 0.015779754
step 0: loss =

test_loss: 0.010153398
step 0: loss = 0.0096
test_loss: 0.009432168
step 0: loss = 0.0089
test_loss: 0.009629483
step 0: loss = 0.0091
test_loss: 0.009806812
step 0: loss = 0.0091
test_loss: 0.010067311
step 0: loss = 0.0097


### Some debugging tests:

In [ ]:
@tf.functionoptimizer.apply_gradients(zip(grads, vars)
def volume_form(x, Omega_Omegabar, mass, restriction):

    kahler_metric = complex_hessian(tf.math.real(model(x)), x)
    volume_form = tf.linalg.det(tf.matmul(restriction, tf.matmul(kahler_metric, restriction, adjoint_b=True)))
    weights = mass / tf.reduce_sum(mass)
    factor = tf.reduce_sum(weights * volume_form / Omega_Omegabar)
    #factor = tf.constant(4.380538, dtype=tf.complex64)
    return  volume_form/factor
    #return factor
for step, (points, Omega_Omegabar, mass, restriction) in enumerate(dataset):
    omega = volume_form(points, Omega_Omegabar, mass, restriction)
    
    weights = mass / tf.reduce_sum(mass)
    print('omega', omega)
    print('OO',Omega_Omegabar)
    print(tf.cast(tf.abs(Omega_Omegabar -  omega), dtype=tf.complex64) / Omega_Omegabar)
   # print(mass/tf.reduce_sum(mass))

In [16]:
for weight in model.trainable_weights:
    print(weight)

<tf.Variable 'Variable:0' shape=(5, 15) dtype=complex64, numpy=
array([[ 2.2768168e+00+1.6352530e+00j,  5.8658237e+00-4.2718673e-01j,
         5.3873414e-01-1.7244281e-01j, -3.9534414e-01+7.2521999e-02j,
        -1.8923687e-02+1.1802919e-02j, -3.8347977e-01-2.2627316e-01j,
        -7.6662725e-01+8.0790210e-01j,  2.6793966e-01+6.7697294e-02j,
        -4.4106621e-01-1.5242220e-01j, -1.6198709e+00-5.7326740e-01j,
         2.8095651e+00+8.1460869e-01j, -1.1343197e+00-2.5883400e+00j,
        -4.6649609e+00+1.1073344e+00j,  1.7060937e-01-2.2714563e-02j,
         8.5827917e-01-7.9530555e-01j],
       [ 1.9180773e+00+4.4432992e-01j, -1.7982172e+00+1.0105304e+00j,
        -6.0589733e+00-8.5629034e-01j,  2.9838247e+00-3.7910095e-01j,
         2.8432586e+00+9.5829725e-01j, -4.8352519e-01-4.6606249e-01j,
        -1.7036586e+00-9.6777737e-01j, -3.2158126e-03+1.6477600e-03j,
         1.7850419e+00+6.1775154e-01j,  4.3784446e-01-8.8729739e-02j,
         2.8238079e-01+1.9228293e-01j, -2.9208440e-01+1.